#### Imports

In [50]:
import pandas as pd
import json
from sqlalchemy import create_engine, Column, Integer, String, Sequence, Float,PrimaryKeyConstraint, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref
from sqlalchemy.sql import *
import pymysql

## Conhecendo os dados

In [2]:
file = open('logs.txt', 'r')
json_list = file.readlines()
file.close()
print(json_list[0])

{"request":{"method":"GET","uri":"\/","url":"http:\/\/yost.com","size":174,"querystring":[],"headers":{"accept":"*\/*","host":"yost.com","user-agent":"curl\/7.37.1"}},"upstream_uri":"\/","response":{"status":500,"size":878,"headers":{"Content-Length":"197","via":"kong\/1.3.0","Connection":"close","access-control-allow-credentials":"true","Content-Type":"application\/json","server":"nginx","access-control-allow-origin":"*"}},"authenticated_entity":{"consumer_id":{"uuid":"72b34d31-4c14-3bae-9cc6-516a0939c9d6"}},"route":{"created_at":1564823899,"hosts":"miller.com","id":"0636a119-b7ee-3828-ae83-5f7ebbb99831","methods":["GET","POST","PUT","DELETE","PATCH","OPTIONS","HEAD"],"paths":["\/"],"preserve_host":false,"protocols":["http","https"],"regex_priority":0,"service":{"id":"c3e86413-648a-3552-90c3-b13491ee07d6"},"strip_path":true,"updated_at":1564823899},"service":{"connect_timeout":60000,"created_at":1563589483,"host":"ritchie.com","id":"c3e86413-648a-3552-90c3-b13491ee07d6","name":"ritchi

In [6]:
data = json.loads(json_list[0])
print('usuario')
print(data["authenticated_entity"]["consumer_id"]["uuid"])
print('----------------------------')
print('serviço')
print(data["service"]["id"])
print(data["service"]["host"])
print(data["service"]["name"])
print('----------------------------')
print('latencias')
print(data["latencies"]["proxy"])
print(data["latencies"]["kong"])
print(data["latencies"]["request"])
print('----------------------------')
print('requisicao')
print(data["request"]["headers"]["host"])
print('----------------------------')
serie = {
    'consumer_id':data["authenticated_entity"]["consumer_id"]["uuid"],
    'service_id':data["service"]["id"],
    'proxy':data["latencies"]["proxy"],
    'kong':data["latencies"]["kong"],
    'request':data["latencies"]["request"],
    'host':data["service"]["host"],
    'name':data["service"]["name"],
    'request_host':data["request"]["headers"]["host"]
    }
print(serie)

usuario
72b34d31-4c14-3bae-9cc6-516a0939c9d6
----------------------------
serviço
c3e86413-648a-3552-90c3-b13491ee07d6
ritchie.com
ritchie
----------------------------
latencias
1836
8
1058
----------------------------
requisicao
yost.com
----------------------------
{'consumer_id': '72b34d31-4c14-3bae-9cc6-516a0939c9d6', 'service_id': 'c3e86413-648a-3552-90c3-b13491ee07d6', 'proxy': 1836, 'kong': 8, 'request': 1058, 'host': 'ritchie.com', 'name': 'ritchie', 'request_host': 'yost.com'}


In [7]:
df = pd.DataFrame(columns = ['consumer_id','service_id','proxy','kong','request','host','name','request_host'])
df = df.append(serie,ignore_index=True)
df.head()

,consumer_id,service_id,proxy,kong,request,host,name,request_host
0,72b34d31-4c14-3bae-9cc6-516a0939c9d6,c3e86413-648a-3552-90c3-b13491ee07d6,1836,8,1058,ritchie.com,ritchie,yost.com


## Extraindo dados relevantes

In [38]:
%%time
df = pd.DataFrame(columns = ['consumer_id','service_id','proxy','kong','request','host','name','request_host'])
serie_list = []
for json_str in json_list:
    data = json.loads(json_str)
    serie = {
    'consumer_id':data["authenticated_entity"]["consumer_id"]["uuid"],
    'service_id':data["service"]["id"],
    'proxy':data["latencies"]["proxy"],
    'kong':data["latencies"]["kong"],
    'request':data["latencies"]["request"],
    'host':data["service"]["host"],
    'name':data["service"]["name"],
    'request_host':data["request"]["headers"]["host"]
    }
    serie_list.append(serie)
    
df = df.append(serie_list,ignore_index=True)
print(len(df))
df.index += 1
df.head()

100000
Wall time: 4.59 s


,consumer_id,service_id,proxy,kong,request,host,name,request_host
1,72b34d31-4c14-3bae-9cc6-516a0939c9d6,c3e86413-648a-3552-90c3-b13491ee07d6,1836,8,1058,ritchie.com,ritchie,yost.com
2,f643db14-a82d-30c2-8c13-889db1d0fcc2,d035ffcf-914a-3007-b028-ae18f04d75b4,1727,10,2185,terry.org,terry,labadie.com
3,beceaa24-823b-3bf9-9ae6-c8dada26b264,a5bf08bd-c030-30d5-8009-83a8c30103bf,896,9,2129,orn.com,orn,wisozk.biz
4,7ba24e1f-ed19-31b2-a4be-4114721d63af,22f8e3a6-01f7-3264-b4b5-9d178df11d06,1209,12,1274,sauer.com,sauer,veum.org
5,1696f3c4-7732-38f2-9ba8-0eeca8df05d7,22f8e3a6-01f7-3264-b4b5-9d178df11d06,1708,20,2244,sauer.com,sauer,ledner.com


## Aplicando transformações nos dados

### Tabela de serviços

In [39]:
servicos = df.drop_duplicates(subset=['service_id']).reset_index(drop=True)\
.drop(['consumer_id','proxy','kong','request','request_host'], axis=1)
servicos.index += 1
servicos_map = servicos['service_id'].to_dict()
servicos_inv_map = {v: k for k, v in servicos_map.items()}
servicos.head()

{'c3e86413-648a-3552-90c3-b13491ee07d6': 1, 'd035ffcf-914a-3007-b028-ae18f04d75b4': 2, 'a5bf08bd-c030-30d5-8009-83a8c30103bf': 3, '22f8e3a6-01f7-3264-b4b5-9d178df11d06': 4, 'eb1ce287-5797-3e45-b9a6-e6d51691257e': 5}


,service_id,host,name
1,c3e86413-648a-3552-90c3-b13491ee07d6,ritchie.com,ritchie
2,d035ffcf-914a-3007-b028-ae18f04d75b4,terry.org,terry
3,a5bf08bd-c030-30d5-8009-83a8c30103bf,orn.com,orn
4,22f8e3a6-01f7-3264-b4b5-9d178df11d06,sauer.com,sauer
5,eb1ce287-5797-3e45-b9a6-e6d51691257e,corkery.com,corkery


### Tabela de Consumidores

In [41]:
costumer = df.drop_duplicates(subset=['consumer_id']).reset_index(drop=True)\
.drop(['service_id','proxy','kong','request','host','name','request_host'], axis=1)
costumer.index += 1
print(len(costumer))
costumer_map = costumer['consumer_id'].to_dict()
costumer_inv_map = {v: k for k, v in costumer_map.items()}
#print(costumer_inv_map)
costumer.head()

9999


,consumer_id
1,72b34d31-4c14-3bae-9cc6-516a0939c9d6
2,f643db14-a82d-30c2-8c13-889db1d0fcc2
3,beceaa24-823b-3bf9-9ae6-c8dada26b264
4,7ba24e1f-ed19-31b2-a4be-4114721d63af
5,1696f3c4-7732-38f2-9ba8-0eeca8df05d7


### Tabela de Requisições

In [43]:
%%time
requisicoes = df.replace({"consumer_id": costumer_inv_map, "service_id": servicos_inv_map})\
.drop(['host','name'], axis=1)
print(len(costumer))
requisicoes.head()

9999
Wall time: 1min 14s


,consumer_id,service_id,proxy,kong,request,request_host
1,1,1,1836,8,1058,yost.com
2,2,2,1727,10,2185,labadie.com
3,3,3,896,9,2129,wisozk.biz
4,4,4,1209,12,1274,veum.org
5,5,4,1708,20,2244,ledner.com


## Carregando dados no banco

### Criando tabelas no sevidor

In [105]:
sqlEngine = create_engine('mysql+pymysql://test:@127.0.0.1:3306/test', pool_recycle=3600)
dbConnection = sqlEngine.connect()
Base = declarative_base()
class Requests(Base):
    __tablename__ = "requests"
    index = Column(Integer, Sequence('index_seq'), primary_key=True)
    consumer_id = Column(Integer)
    service_id = Column(Integer)
    proxy = Column(Integer)
    kong = Column(Integer)
    request = Column(Integer)
    request_host = Column(String(100))
    
class Consumers(Base):
    __tablename__="consumers"
    index =Column(Integer, Sequence('index_seq'), primary_key=True)
    consumer_id = Column(String(200))
    
class Services(Base):
    __tablename__="services"
    index= Column(Integer, Sequence('index_seq'), primary_key=True)
    service_id = Column(String(200))
    host = Column(String(100))
    name = Column(String(100))

Services.__table__.create(bind=sqlEngine, checkfirst=True) 
Requests.__table__.create(bind=sqlEngine, checkfirst=True)
Consumers.__table__.create(bind=sqlEngine, checkfirst=True)
dbConnection.close()

In [107]:
try:
    sqlEngine = create_engine('mysql+pymysql://test:@127.0.0.1:3306/test', pool_recycle=3600)
    dbConnection = sqlEngine.connect()
    frame = requisicoes.to_sql("requests", dbConnection, if_exists='append');
    frame = costumer.to_sql("consumers", dbConnection, if_exists='append')
    frame = servicos.to_sql("services", dbConnection, if_exists='append')
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Tabelas criadas");   
finally:
    dbConnection.close()

Tabelas criadas
